In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf().setMaster("local").setAppName("spark_sql")

In [3]:
# 스파크 context 생성
spark = SparkContext(conf=conf)

In [6]:
spark.getConf().getAll()

[('spark.master', 'local'),
 ('spark.driver.host', '172.30.1.27'),
 ('spark.app.startTime', '1655371188459'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.name', 'spark_sql'),
 ('spark.app.id', 'local-1655371189740'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '60471')]

In [8]:
ticker = spark.parallelize([
    (1, ("Google", "GOOGL", "USA")),
    (2, ("Netflix", "NFLX", "USA")),
    (3, ("Amazon", "AMZN", "USA")),
    (4, ("Tesla", "TSLA", "USA")),
    (5, ("Samsung", "005930", "Korea")),
    (6, ("Kakao", "035720", "Korea")),
])

In [9]:
prices = spark.parallelize([
    (1, (2984, "USA")),
    (2, (645, "USA")),
    (3, (3518, "USA")),
    (4, (1222, "USA")),
    (5, (70600, "USA")),
    (6, (125000, "USA")),
])

In [10]:
ticker, prices

(ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274,
 ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:274)

In [15]:
sql_sc = SQLContext(spark)

/Users/devkhk/opt/anaconda3/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [11]:
## 2000 달러 이상 미국 주식 가지고 오기

# case1 : join 후 filter

ticker_price = ticker.join(prices)

In [24]:
ticker_price.filter(lambda item : item[1][0][2] == "USA" and item[1][1][0] >= 2000).collect()

[(1, (('Google', 'GOOGL', 'USA'), (2984, 'USA'))),
 (3, (('Amazon', 'AMZN', 'USA'), (3518, 'USA')))]

In [35]:
# case 2 filter 후 join

ticker_filter = ticker.filter(lambda x: x[1][2] == "USA")
price_filter = prices.filter(lambda x: x[1][0] >= 2000)

ticker_filter.join(price_filter).collect()

[(1, (('Google', 'GOOGL', 'USA'), (2984, 'USA'))),
 (3, (('Amazon', 'AMZN', 'USA'), (3518, 'USA')))]

## Spark SQL 을 사용하는 이유

- 데이터가 정형화 되어 있는 경우 스파크 sql을 이용하면 위처럼 개발자의 코드에 따라 성능이 들쑥 날쑥하다.
- 스파크 sql은 정형화 되어 있는 데이터를 가장 빠르게 처리 할 수 있게끔 최적화 되어있다.

## 스파크 sql 사용 목적
- 스파크 프로그래밍 내부에서 관계형 데이터 처리를 위해
- 스키마의 정보를 이용해 자동으로 최적화 하기 위해
- 외부 데이터셋을 사용하기 쉽게 하기 위해
